In [2]:
import json
import urllib
import heapq
import time
import os
from pprint import pprint
import hashlib

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv('data.csv')
df

,apkPublishTime,apkUrl,appAverageRating,appDownCount,appId,appName,appRating1,appRating2,appRating3,appRating4,appRating5,appRatingCount,appTags,authorId,authorName,averageRating,categoryId,categoryName,description,editorIntro,fileSize,flag,iconUrl,newFeature,numImages,numSnapshot,pkgName,versionCode,versionName
0,1551261463,https://imtt.dd.qq.com/16891/68CF64B59AB1C0719...,3.533419,7718249331,10910,微信,120054,36772,62098,81567,242469,542960,NaN,179,腾讯公司,3.533419,106,社交,1.可以发语音、文字消息、表情、图片、视频30M流量可以收发上千条语音，省电省流量 2.朋...,NaN,110041256,16533,https://pp.myapp.com/ma_icon/0/icon_10910_1551...,本次更新: - 解决了一些已知问题。 最近更新: - 用「视频动态」，记录眼前的...,5,5,com.tencent.mm,1400,7.0.3
1,1552889019,https://imtt.dd.qq.com/16891/62EAF43D783D22F2E...,4.631833,51811733,42270467,企业微信,85,17,22,23,1097,1244,NaN,179,腾讯公司,4.631833,113,办公,企业微信，是腾讯微信团队为企业打造的高效办公平台。与微信一致的沟通体验，丰富的OA应用，和连...,NaN,101092695,16533,https://pp.myapp.com/ma_icon/0/icon_42270467_1...,2.7.6 新功能介绍 1. 更方便地添加微信联系人 在微信中收到了好友申请，可将他...,5,5,com.tencent.wework,8926,2.7.6
2,1552875740,https://imtt.dd.qq.com/16891/F7A2865DD2ABD817E...,4.847615,41275691,12144239,微信读书,204,63,246,1078,15307,16898,NaN,179,腾讯公司,4.847615,102,阅读,爱读书的人都在用的APP，为什么？ 看书时长可兑换书币，每周推出免费领取无限会员卡免费读全...,好看小说、全本小说，全都免费!,26088353,16533,https://pp.myapp.com/ma_icon/0/icon_12144239_1...,3.4.0 版本更新 1 新加入的微信朋友，须手动关注你后才可以查看你的读书动态 2...,5,5,com.tencent.weread,10135763,3.4.0
3,1552698457,https://imtt.dd.qq.com/16891/BA2997E899F656D77...,4.800338,76090665,12071517,映客,746,185,821,1907,31841,35500,NaN,2068723,北京蜜莱坞网络科技有限公司,4.800338,103,视频,映客7.0，首页全新升级！我想和你在映客不期而遇。 映客，高颜值直播交友平台。直播、短视频...,随时随地，遇见心动,65570916,16533,https://pp.myapp.com/ma_icon/0/icon_12071517_1...,加入你喜欢的主播真爱团，领取主播的专属福利！ 大王卡免流服务上线，随时随地，观看精彩直播；...,5,5,com.meelive.ingkee,1920,7.0.40
4,1525953041,https://imtt.dd.qq.com/16891/3FEF44F4FA542C35A...,4.085141,32382505,10140875,微信电话本,1320,262,600,1126,5865,9173,NaN,179,腾讯公司,4.085141,116,通讯,微信电话本是腾讯出品的一款高清免费通话软件！ 打电话，用微信电话本，边聊边拍！ 微信公众...,腾讯出品的一款高清免费通话软件,33594008,16533,https://pp.myapp.com/ma_icon/0/icon_10140875_1...,1、新增AR通话，自拍生成3D人偶，虚拟与现实结合，超真实通话体验（仅Android 4.4...,5,5,com.tencent.pb,5090,4.5.0
5,1547117130,https://imtt.dd.qq.com/16891/FFD73452D86D9BC68...,3.490401,26220087,28262,腾讯文件-微信QQ文件极速查找,285,78,120,116,547,1146,NaN,179,腾讯公司,3.490401,117,系统,产品介绍： 微信、QQ文件快速分类，极速查找 王卡使用全免流 我们的官方群号：7137...,微信、QQ文件快速分类，极速查找,10378458,16533,https://pp.myapp.com/ma_icon/0/icon_28262_1547...,新版本特性： 1、 支持微信语音识别与保存，重要的声音永久保存。 2、 聊天精选图片...,5,5,com.tencent.FileManager,4710021,4.7.1.0021
6,1550060089,https://imtt.dd.qq.com/16891/90F94E822C053407B...,4.946413,5577146,52633699,微粉管家,25,6,1,4,2278,2314,NaN,18012382,深圳市华深创技术有限公司,4.946413,115,工具,微粉管家是一款超好用的微信好友管理工具， 集一键点赞、群发、清粉于一身，让微信的操作批量自动...,一键清粉点赞群发的微信管家,10635951,153,https://pp.myapp.com/ma_icon/0/icon_52633699_1...,1.优化部分功能，让您使用更便捷 ； 2.修复已知BUG；,5,5,com.felix.clearfans,122,1.2.2
7,1552729022,https://imtt.dd.qq.com/16891/85E890939FA2C32C0...,4.867888,3670725,42349998,定位精灵,60,7,4,5,1998,2074,NaN,17809904,湖南机友科技有限公司,4.867888,115,工具,定位精灵，支持在手机上虚拟位置信息，让您想去哪就去哪，还可以充分利用附近人的功能，让您足不出...,手机定位虚拟位置修改,11097590,153,https://pp.myapp.com/ma_icon/0/icon_42349998_1...,更新说明 1，修复部分功能无法使用BUG 2，优化定位流程，让您使用更便捷,4,4,com.felix.mocklocation,133,1.3.3
8,1552474533,https://imtt.dd.qq.com/16891/D70330F57A1666C9F...,3.433033,8868016334,6633,QQ,198416,54537,82316,94039,346679,775987,NaN,179,腾讯公司,3.433033,106,社交,-----QQ•乐在沟通----- √服务超过90%的移动互联网用户 √多人视频、文...,乐在沟通19年，聊天欢乐9亿人！,60403413,16533,https://pp.myapp.com/ma_icon/0/icon_6633_15524...,- 与好友聊天时，收发的文件分类新增腾讯文档，在线协作更高效； - 双人视频魔法贴纸，魔法...,4,4,com.tencent.mobileqq,1010,7.9.9
9,1552616530,https://imtt.dd.qq.com/16891/81F5E77D038938950...,4.542486,741850296,97949,欢乐斗地主（腾讯）,3801,748,1570,6285,46333,58737,NaN,179,腾讯公司,4.542486,148,棋牌中心,经典棋牌品牌，腾讯游戏精心打造，与8亿玩家一起《欢乐斗地主》！ 【纯正的经典玩法】经典玩法...,腾讯出品，亲朋好友都在玩！,107203286,16533,https://pp.myapp.com/ma_icon/0/icon_97949_1552...,版本更新内容： 1.聊天系统支持使用聊天气泡，可以在世界频道和向好友秀出最炫酷的气泡装扮啦~...,5,5,com.qqgame.hlddz,201,6.092.001


In [109]:
def get_id_pkg(name):
    row = df[df['appName'] == name]
    appid = row['appId']
    pkg = row['pkgName']
    idx = appid.index.values[0]
    return appid[idx],pkg[idx]

get_id_pkg('新浪微博')

(9926, 'com.sina.weibo')

In [110]:
BASEURL = 'http://mapp.qzone.qq.com/cgi-bin/mapp/mapp_getcomment?type=myapp_all_comment&appid={}&pkgname={}&comment_contex={}&need_score=1&platform=touch'

In [50]:
def query(url):
    resp = ''
    try:
        resp = urllib.request.urlopen(url).read().decode('utf8', 'replace')
    except KeyboardInterrupt:
        raise

    resp = resp[:-1]
    
    j = json.loads(resp)
    return j

query(BASEURL.format(*get_id_pkg('微信'), ''))

{'code': 0,
 'data': {'yyb_comment': {'comment_contex': 'id=1444533652095205376,t=1554977345',
   'comments': [{'brand': '',
     'content': '微信相比QQ有优点也有缺点，优点的话就是隐私度更高，朋友只能看到自己想要展示出来的东西，页面比较整洁，缺点的话需要微信能够出一个自动同步聊天记录的功能，换一个手机或者电脑之前的记录就没有了，虽然可以自己手动迁移，但是有点麻烦。',
     'createdTime': 1528395927,
     'deviceInfo': '',
     'name': '星。空',
     'score': 4,
     'versionCode': 1321},
    {'brand': '',
     'content': '我觉得90后还是用QQ比较多吧，但是因为家人都是用的微信，所以也算得上是微信用户，强烈觉得微信群可以调整一下，之前不知道微信群需要自己保存才能保留，直接删除了对话框之后就找不到了，好多消息都丢失了。',
     'createdTime': 1528806738,
     'deviceInfo': '',
     'name': '小二，上酒',
     'score': 4,
     'versionCode': 1321},
    {'brand': '',
     'content': '做为一个刚上小学的孩子妈妈，微信让我们更能直接的了解到孩子的学习生活，相信大家都有和老师的交流群，微信给大家提供了一个相互交流的方便平台，平时孩子的作业、需要准备的东西老师都是通过微信群发布，不用担心错过什么，与老师交流也不再需要面对面那么麻烦。',
     'createdTime': 1521758588,
     'deviceInfo': '',
     'name': '渡己',
     'score': 5,
     'versionCode': 1321},
    {'brand': '',
     'content': '用户群庞大的一款软件，用微信最多的地方除了聊天就是微信支付吧，线下购物的话微信支付也很方便，现在又发现

In [58]:
def get_comments(appname):
    
    comments = {}
    count = 0
    
    appid, pkg = get_id_pkg(appname)
    context = ''
    
    while True:
        j = query(BASEURL.format(appid, pkg, context))
        
        try:
            context = j['data']['yyb_comment']['comment_contex']
            # id%3D1444533652095205376%2Ct%3D1554977345
            context = context.replace('=', '%3D').replace(',','%2t')
        except:
            return comments  # no next page

        oldlen = count
        
        for c in j['data']['yyb_comment']['comments']:
            comment_id = hash(str(c))
            comments[comment_id] = c
            count += 1
            
        print(count, end=' ')
        time.sleep(.6)
            
    return comments

In [75]:
def write_to_file(d, appname):
    print(f'writing {appname}.csv')
    data = pd.DataFrame.from_dict(d, orient='index')
    data['createdTime'] = pd.to_datetime(data['createdTime'],unit='s')
    data = data.sort_values(by=['createdTime'])
    data.to_csv(f'comments/{appname}.csv',index=False)

In [78]:
write_to_file(d, '微信')

writing 微信.csv


In [112]:
names = [
    '微信',
    'UC浏览器',
    'QQ',
    '360手机卫士',
    '哔哩哔哩',
    '王者荣耀',
    '网易云音乐',
    '新浪微博',
    '大众点评',
    '我的世界：新版本',
    '百度地图',
    '高德地图'
]

for name in names:
    print(name, get_id_pkg(name))

微信 (10910, 'com.tencent.mm')
UC浏览器 (10936, 'com.UCMobile')
QQ (6633, 'com.tencent.mobileqq')
360手机卫士 (2598, 'com.qihoo360.mobilesafe')
哔哩哔哩 (73622, 'tv.danmaku.bili')
王者荣耀 (12127266, 'com.tencent.tmgp.sgame')
网易云音乐 (1168851, 'com.netease.cloudmusic')
新浪微博 (9926, 'com.sina.weibo')
大众点评 (7960, 'com.dianping.v1')
我的世界：新版本 (52497529, 'com.tencent.tmgp.wdsj666')
百度地图 (9997, 'com.baidu.BaiduMap')
高德地图 (7678, 'com.autonavi.minimap')


In [117]:
for name in names:
    print(name)
    d = get_comments(name)
    write_to_file(d, name)

微信
16 26 36 46 56 66 76 86 96 106 116 126 136 146 156 166 176 186 196 206 216 226 236 246 256 266 276 286 296 306 316 326 336 346 356 366 376 386 396 406 416 426 436 446 456 466 476 486 496 506 516 526 536 546 556 566 576 586 596 606 616 626 636 646 656 666 676 686 696 706 716 726 736 746 756 766 776 786 796 806 816 826 836 846 856 866 876 886 896 906 916 926 936 946 956 966 976 986 996 1006 1016 1026 1036 1046 1056 1066 1076 1086 1096 1106 1116 1126 1136 1146 1156 1166 1176 1186 1196 1206 1216 1226 1236 1246 1256 1266 1276 1286 1296 1306 1316 1326 1336 1346 1356 1366 1376 1386 1396 1406 1416 1426 1436 1446 1456 1466 1476 1486 1496 1506 1516 1526 1536 1546 1556 1566 1576 1586 1596 1606 1616 1626 1636 1646 1656 1666 1676 1686 1696 1706 1716 1726 1736 1746 1756 1766 1776 1786 1796 1806 1816 1826 1836 1846 1856 1866 1876 1886 1896 1906 1916 1926 1936 1946 1956 1966 1976 1986 1996 2006 2016 2026 2036 2046 2056 2066 2076 2086 2096 2106 2116 2126 2136 2146 2156 2166 2176 2186 2196 2206 2216 

15 25 35 45 55 65 75 85 95 105 115 125 135 145 155 165 175 185 195 205 215 225 235 245 255 265 275 285 295 305 315 325 335 345 355 365 375 385 395 405 415 425 435 445 455 465 475 485 495 505 515 525 535 545 555 565 575 585 595 605 615 625 635 645 655 665 675 685 695 705 715 725 735 745 755 765 775 785 795 805 815 825 835 845 855 865 875 885 895 905 915 925 935 945 955 965 975 985 995 1005 1015 1025 1035 1045 1055 1065 1075 1085 1095 1105 1115 1125 1135 1145 1155 1165 1175 1185 1195 1205 1215 1225 1235 1245 1255 1265 1275 1285 1295 1305 1315 1325 1335 1345 1355 1365 1375 1385 1395 1405 1415 1425 1435 1445 1455 1464 1474 1484 1494 1504 1514 1524 1534 1544 1554 1564 1574 1584 1594 1604 1614 1624 1634 1644 1654 1664 1674 1684 1694 1704 1714 1724 1734 1744 1754 1764 1774 1784 1794 1804 1814 1824 1834 1844 1854 1864 1874 1884 1894 1904 1914 1924 1934 1944 1954 1964 1974 1984 1994 2004 2014 2024 2034 2044 2054 2064 2074 2084 2094 2104 2114 2124 2134 2144 2154 2164 2174 2184 2194 2204 2214 222

16 26 36 46 56 66 76 86 95 105 115 125 135 145 155 165 175 185 195 205 215 225 235 245 255 265 275 285 295 305 315 325 335 345 355 365 375 385 395 405 415 425 435 445 455 465 475 485 495 505 515 525 535 545 555 564 574 584 594 604 614 624 634 644 654 664 674 684 694 704 714 724 734 744 752 762 772 782 792 802 812 822 832 842 852 862 872 882 892 902 912 922 930 940 950 960 969 979 989 999 1009 1019 1029 1039 1049 1059 1069 1079 1089 1099 1109 1119 1129 1137 1147 1157 1167 1177 1187 1197 1207 1217 1227 1237 1247 1257 1267 1277 1287 1297 1307 1317 1327 1337 1347 1356 1366 1376 1386 1395 1404 1414 1424 1434 1444 1454 1464 1474 1484 1494 1504 1514 1524 1534 1544 1554 1564 1574 1584 1593 1603 1613 1622 1632 1642 1651 1661 1671 1681 1691 1701 1711 1721 1731 1741 1751 1761 1771 1781 1791 1801 1811 1821 1831 1841 1850 1859 1869 1879 1889 1898 1908 1918 1928 1938 1948 1958 1968 1976 1986 1996 2005 2014 2024 2034 2044 2054 2063 2073 2083 2093 2103 2113 2123 2133 2143 2153 2163 2173 2183 2193 2203